In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2,norm,f,f_oneway
from sampling import random_sampling, stratified_sampling

In [2]:
data = pd.read_csv("dataset\heart_2022_cleaned.csv")

In [3]:
num_samples = 100

In [4]:
data.shape

(354862, 40)

In [5]:
def do_chi2_test(data,column1,column2,alpha = 0.01):
    print("***************************************************************************************************************************")
    print("Chi2 Test for ",column1 ," and " ,column2)
    observed={}
    cols=data[column1].unique()
    for i in range(len(data[column2])):
        if data[column2].iloc[i] not in observed:
            observed[data[column2].iloc[i]]={}
            for j in cols:
                observed[data[column2].iloc[i]][j]=0
        observed[data[column2].iloc[i]][data[column1].iloc[i]]+=1

    observed=pd.DataFrame(observed).T
    observed=observed.sort_index()
    observed=observed[sorted(observed.columns)]
    print("Observed")
    print(observed)
    print()
    observed=observed.to_numpy()
    row_sums = observed.sum(axis=1)
    col_sums = observed.sum(axis=0)
    total = row_sums.sum()
    # expected = np.outer(row_sums, col_sums) / total

    expected=np.zeros(observed.shape)
    for i in range(observed.shape[0]):
        for j in range(observed.shape[1]):
            expected[i][j]=row_sums[i]*col_sums[j]/total
    exp=pd.DataFrame(expected)
    exp.columns=sorted(data[column1].unique())
    exp.index=sorted(data[column2].unique())
    print("Expected") 
    print(exp)
    chi2_stat = ((observed - expected)**2 / expected).sum()
   
    dof=(len(row_sums)-1)*(len(col_sums)-1)
    critical = chi2.ppf(1-alpha, dof)
    print("chi2_stat: ",chi2_stat)
    print("critical: ",critical)
    if chi2_stat>=critical:
        print("Dependent (reject H0)")
    else:
        print("Independent (fail to reject H0)")
    print()

### $\chi^2$ Test for independence between age category and Heart Health ###

In [6]:
categorical_columns = data.select_dtypes(include=['object']).columns
categorical_columns

Index(['State', 'Sex', 'GeneralHealth', 'LastCheckupTime',
       'PhysicalActivities', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'AlcoholDrinkers', 'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver',
       'TetanusLast10Tdap', 'HighRiskLastYear', 'CovidPos'],
      dtype='object')

In [7]:
for col in categorical_columns: 
    if col != 'GeneralHealth':
        do_chi2_test(data,col,'GeneralHealth',0.01)

***************************************************************************************************************************
Chi2 Test for  State  and  GeneralHealth
Observed
           Alabama  Alaska  Arizona  Arkansas  California  Colorado  \
Excellent      470     765     1279       469        1555      1473   
Fair           705     552     1259       766        1106       786   
Good          1231    1559     2554      1383        2657      2245   
Poor           215     216      411       335         323       251   
Very good     1056    1621     2699      1185        2861      2847   

           Connecticut  Delaware  District of Columbia  Florida  ...  \
Excellent         1454       501                   589     1671  ...   
Fair               841       419                   259     1499  ...   
Good              2157      1011                   739     3262  ...   
Poor               228       109                    55      573  ...   
Very good         2687      1077       

### Z-test for proportion ###

In [8]:
def do_proportion_test(data,column,threshold,alpha,po=0.5):
    p = (data[column]>threshold).sum()/len(data[column])
    z = (p-po)/np.sqrt(po*(1-po)/len(data[column]))
    print("z stat : ",z)
    critical_value = norm.ppf(1-alpha)
    print("critical_value : ",critical_value)

    if z<=critical_value:
        print("Reject H0")
    else:
        print("Fail to reject H0")

In [9]:
do_proportion_test(data,"BMI",25,0.05,0.4)  

z stat :  350.39893296453636
critical_value :  1.6448536269514722
Fail to reject H0


### One-Way ANOVA TEST ###

In [10]:
def perform_anova_test(data,column):
    grouped_data = [group[column].values for name,group in data.groupby('GeneralHealth')]   
    x_bars = [np.mean(group) for group in grouped_data]
    print(x_bars)
    x_bar_ka_bar = np.mean(x_bars)
    print(x_bar_ka_bar)
    SSC = sum([len(group)*(x_bar-x_bar_ka_bar)**2 for group,x_bar in zip(grouped_data,x_bars)])
    SSE = sum([sum((group-x_bar)**2) for group,x_bar in zip(grouped_data,x_bars)])
    n1 = 4
    n2 = len(data[column])-n1
    MSC = SSC/n1
    MSE = SSE/n2
    f_stat = MSC/MSE
    f_critical = f.ppf(0.95,n1,n2)
    print("f_stat : ",f_stat)
    print("f_critical : ",f_critical)
    if f_stat<f_critical:
        print("Fail to reject H0")
    else:
        print("Reject H0")



In [11]:
grouped_data = perform_anova_test(data,'BMI')
grouped_data

[26.021939921392473, 30.76459656222621, 29.52744049036469, 30.557551853772363, 27.597127288039037]
28.893731223158955
f_stat :  5849.72488967869
f_critical :  2.37195728399616
Reject H0
